In [1]:
import pandas as pd
import json as js

with open("/u/pmpande/author_profiling/Computational-Personality-Recognition/fb.json") as f:
    data_fb = pd.DataFrame(js.loads(line) for line in f)
f.close()

print(len(data_fb))

9917


In [2]:
data_fb.drop(data_fb.head(1).index, inplace=True)

In [3]:
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from empath import Empath
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

accuracy_DT = 0.0
rmse_DT = [0.0, 0.0, 0.0, 0.0, 0.0]
h_loss_DT = 0.0

accuracy_RF = 0.0
rmse_RF = [0.0, 0.0, 0.0, 0.0, 0.0]
h_loss_RF = 0.0
for i in range(10):
    rand_fb = []
    temp_data = data_fb.copy()
    rand_fb = random.sample(range(1,len(temp_data)-1),1000)
    test = temp_data.iloc[rand_fb].copy()
    train = temp_data.drop(rand_fb)
    train.drop(train.columns[[1,7,8,9,10,11]], axis=1, inplace=True)
    test.drop(test.columns[[1,7,8,9,10,11]], axis=1, inplace=True)
    
    corpus = []
    truth = []
    for row in train.iterrows():
        Y = [0,0,0,0,0]
        corpus.append(row[1]['STATUS'])
        if row[1]['AGR'] == 'y': Y[0] = 1
        if row[1]['CON'] == 'y': Y[1] = 1
        if row[1]['EXT'] == 'y': Y[2] = 1
        if row[1]['NEU'] == 'y': Y[3] = 1
        if row[1]['OPN'] == 'y': Y[4] = 1
        truth.append(Y)
    truth = np.array(truth)    
    lexicon = Empath()
    feat = []
    a = {}
    for row in corpus:
        a = lexicon.analyze(row, normalize=True)
        temp = []
        for k,v in a.items():
            temp.append(v)
        feat.append(temp)
    feat = np.array(feat)
    
    feat_norm = normalize(feat, axis=1, norm='l1')
    clf_DT = tree.DecisionTreeClassifier()
    clf_DT = clf_DT.fit(feat_norm, truth)
    
    clf_RF = RandomForestClassifier()
    clf_RF = clf_RF.fit(feat_norm, truth)
    
    test_data = []
    truth_test = []
    for row in test.iterrows():
        Y = [0,0,0,0,0]
        test_data.append(row[1]['STATUS'])
        if row[1]['AGR'] == 'y': Y[0] = 1
        if row[1]['CON'] == 'y': Y[1] = 1
        if row[1]['EXT'] == 'y': Y[2] = 1
        if row[1]['NEU'] == 'y': Y[3] = 1
        if row[1]['OPN'] == 'y': Y[4] = 1
        truth_test.append(Y)
    truth_test = np.array(truth_test)    
    test_feat = []
    a_test = {}
    for row in test_data:
        a_test = lexicon.analyze(row, normalize=True)
        temp = []
        for k,v in a_test.items():
            temp.append(v)
        test_feat.append(temp)
    test_feat = np.array(test_feat)
    test_feat_norm = normalize(test_feat, axis=1, norm='l1')
    
    print("----------------- Iteration : " + str(i) + "-----------------")
    print("Results for DT")
    output_DT = clf_DT.predict(test_feat_norm)
    acc = accuracy_score(truth_test, output_DT, normalize=True)
    loss = hamming_loss(truth_test, output_DT)
    err = [sqrt(mean_squared_error(truth_test[:,i], output_DT[:,i])) for i in range(len(truth_test[0,:]))] 
    accuracy_DT += acc
    h_loss_DT += loss
    print("Accuracy : " + str(acc))
    for per_type in range(len(rmse_DT)):
        rmse_DT[per_type] += err[per_type]
    print("RMSE - AGR for interation " + str(i) + ": " + str(err[0]))
    print("RMSE - CON for interation " + str(i) + ": " + str(err[1]))
    print("RMSE - EXT for interation " + str(i) + ": " + str(err[2]))
    print("RMSE - NEU for interation " + str(i) + ": " + str(err[3]))
    print("RMSE - OPN for interation " + str(i) + ": " + str(err[4]))
    print("Hamming loss : " + str(loss))
    print("")
    print("Results for RF")
    output_RF = clf_RF.predict(test_feat_norm)
    acc = accuracy_score(truth_test, output_RF, normalize=True)
    loss = hamming_loss(truth_test, output_RF)
    err = [sqrt(mean_squared_error(truth_test[:,i], output_RF[:,i])) for i in range(len(truth_test[0,:]))] 
    accuracy_RF += acc
    h_loss_RF += loss
    print("Accuracy : " + str(acc))
    for per_type in range(len(rmse_RF)):
        rmse_RF[per_type] += err[per_type]
    print("RMSE - AGR for interation " + str(i) + ": " + str(err[0]))
    print("RMSE - CON for interation " + str(i) + ": " + str(err[1]))
    print("RMSE - EXT for interation " + str(i) + ": " + str(err[2]))
    print("RMSE - NEU for interation " + str(i) + ": " + str(err[3]))
    print("RMSE - OPN for interation " + str(i) + ": " + str(err[4]))
    print("Hamming loss : " + str(loss))
    del test
    del train
    del temp_data
print("--------------------------------")
print("Results for DT:")
print("Average accuracy is :" + str(accuracy_DT/10))
print("Average RMSE - AGR is " + ": " + str(rmse_DT[0]/10))
print("Average RMSE - CON is " + ": " + str(rmse_DT[1]/10))
print("Average RMSE - EXT is " + ": " + str(rmse_DT[2]/10))
print("Average RMSE - NEU is " + ": " + str(rmse_DT[3]/10))
print("Average RMSE - OPN is " + ": " + str(rmse_DT[4]/10))
print("Average Hamming loss is :" + str(h_loss_DT/10))
print("")
print("Results for RF:")
print("Average accuracy is :" + str(accuracy_RF/10))
print("Average RMSE - AGR is " + ": " + str(rmse_RF[0]/10))
print("Average RMSE - CON is " + ": " + str(rmse_RF[1]/10))
print("Average RMSE - EXT is " + ": " + str(rmse_RF[2]/10))
print("Average RMSE - NEU is " + ": " + str(rmse_RF[3]/10))
print("Average RMSE - OPN is " + ": " + str(rmse_RF[4]/10))
print("Average Hamming loss is :" + str(h_loss_RF/10))

----------------- Iteration : 0-----------------
Results for DT
Accuracy : 0.463
RMSE - AGR for interation 0: 0.5300943312279429
RMSE - CON for interation 0: 0.48373546489791297
RMSE - EXT for interation 0: 0.4827007354458868
RMSE - NEU for interation 0: 0.4636809247747852
RMSE - OPN for interation 0: 0.3807886552931954
Hamming loss : 0.2216

Results for RF
Accuracy : 0.448
RMSE - AGR for interation 0: 0.5300943312279429
RMSE - CON for interation 0: 0.498998997994986
RMSE - EXT for interation 0: 0.4939635614091388
RMSE - NEU for interation 0: 0.4753945729601885
RMSE - OPN for interation 0: 0.3847076812334269
Hamming loss : 0.2296
----------------- Iteration : 1-----------------
Results for DT
Accuracy : 0.476
RMSE - AGR for interation 1: 0.48887626246321264
RMSE - CON for interation 1: 0.4949747468305833
RMSE - EXT for interation 1: 0.4658325879540846
RMSE - NEU for interation 1: 0.4669047011971501
RMSE - OPN for interation 1: 0.38340579025361626
Hamming loss : 0.2132

Results for RF
A

In [4]:
import pandas as pd
import json as js

with open("/u/pmpande/author_profiling/Computational-Personality-Recognition/essays.json") as f:
    data_essays = pd.DataFrame(js.loads(line) for line in f)
f.close()

print(len(data_essays))

2469


In [5]:
data_essays.drop(data_essays.head(1).index, inplace=True)

In [6]:
test_essays = data_essays.copy()
test_essays.drop(data_essays.columns[[1]], axis=1, inplace=True)

corpus_essays = []
truth_essays = []
for row in test_essays.iterrows():
    Y = [0,0,0,0,0]
    corpus_essays.append(row[1]['STATUS'])
    if row[1]['AGR'] == 'y': Y[0] = 1
    if row[1]['CON'] == 'y': Y[1] = 1
    if row[1]['EXT'] == 'y': Y[2] = 1
    if row[1]['NEU'] == 'y': Y[3] = 1
    if row[1]['OPN'] == 'y': Y[4] = 1
    truth_essays.append(Y)
truth_essays = np.array(truth_essays)
feat_essays = []
a = {}
for row in corpus_essays:
    a = lexicon.analyze(row, normalize=True)
    temp = []
    for k,v in a.items():
        temp.append(v)
    feat_essays.append(temp)
feat_essays = np.array(feat_essays)
feat_essays_norm = normalize(feat_essays, axis=1, norm='l1')

print("Results for DT")
output_essays_DT = clf_DT.predict(feat_essays_norm)

print("Accuracy : " + str(accuracy_score(truth_essays, output_essays_DT, normalize=True)))
print("Hamming Loss : " + str(hamming_loss(truth_essays, output_essays_DT)))
err = [sqrt(mean_squared_error(truth_essays[:,i], output_essays_DT[:,i])) for i in range(len(truth_essays[0,:]))]
print("RMSE - AGR for interation " + str(i) + ": " + str(err[0]))
print("RMSE - CON for interation " + str(i) + ": " + str(err[1]))
print("RMSE - EXT for interation " + str(i) + ": " + str(err[2]))
print("RMSE - NEU for interation " + str(i) + ": " + str(err[3]))
print("RMSE - OPN for interation " + str(i) + ": " + str(err[4]))

print("Results for RF")
output_essays_RF = clf_RF.predict(feat_essays_norm)

print("Accuracy : " + str(accuracy_score(truth_essays, output_essays_RF, normalize=True)))
print("Hamming Loss : " + str(hamming_loss(truth_essays, output_essays_RF)))
err = [sqrt(mean_squared_error(truth_essays[:,i], output_essays_RF[:,i])) for i in range(len(truth_essays[0,:]))]
print("RMSE - AGR for interation " + str(i) + ": " + str(err[0]))
print("RMSE - CON for interation " + str(i) + ": " + str(err[1]))
print("RMSE - EXT for interation " + str(i) + ": " + str(err[2]))
print("RMSE - NEU for interation " + str(i) + ": " + str(err[3]))
print("RMSE - OPN for interation " + str(i) + ": " + str(err[4]))

Results for DT
Accuracy : 0.041734197731
Hamming Loss : 0.49846029173419776
RMSE - AGR for interation 9: 0.7053856262255245
RMSE - CON for interation 9: 0.6935100863508333
RMSE - EXT for interation 9: 0.7159335010724533
RMSE - NEU for interation 9: 0.7025076638928581
RMSE - OPN for interation 9: 0.7125296775075487
Results for RF
Accuracy : 0.0336304700162
Hamming Loss : 0.4992706645056726
RMSE - AGR for interation 9: 0.723253499118417
RMSE - CON for interation 9: 0.7027959904651836
RMSE - EXT for interation 9: 0.7056727769096047
RMSE - NEU for interation 9: 0.7079657895387406
RMSE - OPN for interation 9: 0.6929255855421024
